## Unit 4 | Assignment - Pandas, Pandas, Pandas

## Background

The data dive continues!

## Option 1: Heroes of Pymoli

![Fantasy](Images/Fantasy.jpg)

Congratulations! After a lot of hard work in the data munging mines, you've landed a job as Lead Analyst for an independent gaming company. You've been assigned the task of analyzing the data for their most recent fantasy game Heroes of Pymoli. 

Like many others in its genre, the game is free-to-play, but players are encouraged to purchase optional items that enhance their playing experience. As a first task, the company would like you to generate a report that breaks down the game's purchasing data into meaningful insights.

In [93]:
!pwd

/Users/pooya/Documents/DataScienceCource/homeworks/04-pandas


In [94]:
import os
import json
import pandas as pd
import numpy as np

In [95]:
path = "./HeroesOfPymoli/purchase_data.json"

In [166]:
with open(path, 'r') as file:
    purchase_data = pd.DataFrame(json.load(file))
purchase_data.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


Your final report should include each of the following:

**Player Count**

* Total Number of Players

In [97]:
players_count = purchase_data["SN"].unique().size
players_count

573

**Purchasing Analysis (Total)**

* Number of Unique Items

In [140]:
items_count = purchase_data["Item ID"].unique().size
items_count

183

* Average Purchase Price

In [99]:
average_purchace_price = purchase_data["Price"].mean()
average_purchace_price

2.931192307692303

* Total Number of Purchases

In [100]:
purchase_count = purchase_data["SN"].size
purchase_count

780

* Total Revenue

In [101]:
total_revenue = purchase_data["Price"].sum()
total_revenue

2286.3299999999963

**Gender Demographics**

* Percentage and Count of Male Players

In [179]:
count_of_gender = purchase_data[["Gender", "SN", "Item ID"]].groupby(["SN", "Gender"]).count().groupby(["Gender"]).count()
count_of_gender

,Item ID
Gender,
Female,100
Male,465
Other / Non-Disclosed,8


* Percentage and Count of Female Players

In [ ]:
percentage_of_males = count_of_males / 

* Percentage and Count of Other / Non-Disclosed

**Purchasing Analysis (Gender)** 

* The below each broken by gender
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Normalized Totals

**Age Demographics**

* The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.) 
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Normalized Totals

**Top Spenders**

* Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
  * SN
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value

**Most Popular Items**

* Identify the 5 most popular items by purchase count, then list (in a table):
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value

**Most Profitable Items**

* Identify the 5 most profitable items by total purchase value, then list (in a table):
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value

As final considerations:

* Your script must work for both data-sets given.
* You must use the Pandas Library and the Jupyter Notebook.
* You must submit a link to your Jupyter Notebook with the viewable Data Frames. 
* You must include an exported markdown version of your Notebook called  `README.md` in your GitHub repository.  
* You must include a written description of three observable trends based on the data. 
* See [Example Solution](HeroesOfPymoli/HeroesOfPymoli_Example.pdf) for a reference on expected format. 